In [291]:
from pydantic_ai import Agent
from dotenv import load_dotenv
import nest_asyncio


nest_asyncio.apply()
_ = load_dotenv()

In [28]:
agent = Agent('gemini-1.5-flash', system_prompt="You are a personal assistant, you answer questions in one sentence.")


In [29]:
result = agent.run_sync('What is pydantic?')

In [30]:
result.cost()

Cost(request_tokens=21, response_tokens=15, total_tokens=36, details=None)

In [2]:
from pydantic import BaseModel
# Structured response
class MyModel(BaseModel):
    city: str
    country: str

structured_agent = Agent('gemini-1.5-flash', result_type=MyModel)

res = structured_agent.run_sync('What is the most populated city in europe?')


C:\Users\milia.io\AppData\Local\Temp\ipykernel_28928\4257384914.py:9: LogfireNotConfiguredWarning: No logs or spans will be created until `logfire.configure()` has been called. Set the environment variable LOGFIRE_IGNORE_NO_CONFIG=1 or add ignore_no_config=true in pyproject.toml to suppress this warning.
  res = structured_agent.run_sync('What is the most populated city in europe?')


In [32]:
res.data

MyModel(city='London', country='UK')

In [3]:
from pydantic_ai import Tool, RunContext

def add(a: int, b: int):
    return a+b

add_agent = Agent('gemini-1.5-flash', tools=[Tool(add, takes_ctx=False)], system_prompt="You are an addition assistant. You can only add numbers."
)

result = add_agent.run_sync('What is 2*4')




In [4]:
print(result.data)

I can only add numbers.  I cannot perform multiplication.



In [5]:
result = add_agent.run_sync('What is 2+4')


In [6]:
print(result.data)

6



In [ ]:
todo_list = []

In [336]:
def append_todo_list(item: str) -> str:
    """Appends an item to the todo list."""
    print('Appending', item)
    with open("test.txt", "a") as myfile:
        myfile.write(item + '\n')

    return 'Success'

def update_todo_list(item: str) -> str:
    """Updates a current item in the todo list"""
    print('Updating to todo list', item)
    return 'Success'

def get_todo_list() -> str:
    """Returns the current todo list"""
    print('Getting todo list')
    res = ""
    with open("test.txt", "r") as myfile:
        res = myfile.read()
    return res

In [ ]:
agent = Agent('gemini-1.5-flash',
              system_prompt="""
                        You are a todo list manager. You help the user manage their todo list.
                        You can fetch, update and append the user's todo list.
                        First fetch the todo list, then analzye the user's request.
                        Create a plan for the steps required to help the user. Fetch the todo list fisrt to help you plan.
                        Don't append to the list if the user's request is already on the todo list. 
                        """,
                         tools=[Tool(get_todo_list, takes_ctx=False),Tool(append_todo_list, takes_ctx=False), Tool(update_todo_list, takes_ctx=False)])

In [353]:
result = agent.run_sync("I need to pack by bags for my flight tomorrow. I need to cook dinner later. I need to set my alarm for 5am.")

In [354]:
result.data

'Your to do list already contains these items. Is there anything else I can help with?'

In [161]:
checker_result = agent_checker.run_sync(f"""Check this request. 
                                Here is the user's current todo list: 
                                Todo:
                                    - Make dinner
                                        
                                The assistant's plan is {result.data}
                                """)

In [162]:
checker_result.data

'The assistant\'s plan is insufficient. While adding "Cook dinner" is a step in the right direction, it\'s too vague.  To make a dinner plan more helpful, the assistant needs more information.  Here\'s feedback to give the assistant:\n\n"The plan to add \'Cook dinner\' is a start, but lacks crucial detail. To improve, the assistant should:\n\n1. **Clarify the specifics of dinner:**  Ask the user what they want to eat.  If the user doesn\'t have a preference, suggest some options based on available ingredients or dietary restrictions.  The plan should include a specific recipe or at least a list of ingredients needed.\n\n2. **Break down the task:** Cooking dinner isn\'t a single action.  It involves gathering ingredients, preparing ingredients (chopping, washing), cooking, and cleaning up. The assistant should break this down into smaller, manageable tasks within the to-do list.  Example:\n\n    * Shop for ingredients (if necessary, including a list)\n    * Prepare vegetables (specify w

In [253]:
result = agent.run_sync('I need to pack by bags for my flight tomorrow. I need to cook dinner later. I need to set my alarm for 5am.', message_history=result.all_messages())

In [254]:
result.data

'The request to add "pack bags for flight tomorrow", "cook dinner later", and "set alarm for 5am" is ignored because these items already exist in the todo list.\n'

In [125]:
print(result.new_messages())

[UserPrompt(content='I need to pack by bags for my flight tomorrow. I need to cook dinner later.', timestamp=datetime.datetime(2024, 12, 8, 20, 42, 49, 632415, tzinfo=datetime.timezone.utc), role='user'), ModelStructuredResponse(calls=[ToolCall(tool_name='append_todo_list', args=ArgsDict(args_dict={'item': 'Pack bags for flight'}), tool_id=None)], timestamp=datetime.datetime(2024, 12, 8, 20, 42, 50, 626532, tzinfo=datetime.timezone.utc), role='model-structured-response'), ToolReturn(tool_name='append_todo_list', content='Success', tool_id=None, timestamp=datetime.datetime(2024, 12, 8, 20, 42, 50, 634571, tzinfo=datetime.timezone.utc), role='tool-return'), ModelTextResponse(content='Todo:\n- Cook dinner\n- Pack bags for flight\n', timestamp=datetime.datetime(2024, 12, 8, 20, 42, 51, 100262, tzinfo=datetime.timezone.utc), role='model-text-response')]
